
# Public Key Signatures
With a public key signature, we use the private key to sign a hash of the message, and then prove the signature with the public key. In this case, Bob signs a hash of the message with his private key, and then Alice will check this with his public key:

<img src='graphics/g_sig_10.png' width="800px">

## RSA Signatures
In the following, we create an RSA signature, using the private key, and then check with the public key:

In [1]:
# https://asecuritysite.com/hazmat/hashnew7
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import exceptions

import sys
import binascii

size=512
message = b"Hello world"

if (len(sys.argv)>1):
	message=str(sys.argv[1]).encode()
if (len(sys.argv)>2):
	size=int(sys.argv[2])

print("Message: ",message)
print("Key size: ",size)
try:
  private_key = rsa.generate_private_key(public_exponent=65537,key_size=size)


  pub = private_key.public_key()



  signature = private_key.sign(message,padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),hashes.SHA256())

  print ("\nSignature: ",binascii.b2a_hex(signature).decode())

  try:
    rtn=pub.verify(signature,message,padding.PSS(mgf=padding.MGF1(hashes.SHA256()),salt_length=padding.PSS.MAX_LENGTH),hashes.SHA256())
  except exceptions.InvalidSignature:
    print("A bad signature failed")
  else:
    print("Good signature verified")


  try:
    pub.verify(signature,b"test",padding.PSS(mgf=padding.MGF1(hashes.SHA256()),salt_length=padding.PSS.MAX_LENGTH),hashes.SHA256())
  except exceptions.InvalidSignature:
    print("A bad signature failed")
  else:
    print("Bad signature verified")



  print ("\nVerified: ",rtn)
  print("\n=== Private Key PEM format ===")
  pem = private_key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())


  print ("\nPrivate key (PEM):\n",pem.decode())


  pem = pub.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo).decode("utf-8")



  print("\n=== Public Key format ===")
  print ("Public key (PEM):\n",pem)


except Exception as e:
    print(e)

Message=Hello
p=1015811308611355393
q=755523213775410319

d=589399628769593821059839430475148801
e=65537
N=767469024471456365658376241308500367

Private key (d,n)
Public key (e,n)

cipher=302667991570245321224673969207643936
decipher=b'Hello'



> Implement the signature with SHA-1 hashing, and verify the operation/

> Change the key size of 2,048 bits, and verify the operation.

## ECDSA
ECDSA has been around for over two decades and was first proposed in [1]. The ECDSA method significantly improved the performance of signing messages than the RSA-based DSA method. Its usage of elliptic curve methods speeded up the whole process and supported much smaller key sizes. Its crown and glory were being selected by Satoshi Nakamoto for his Bitcoin protocol, and then its adoption into Ethereum. 

[1] Johnson, D., Menezes, A., & Vanstone, S. (2001). The elliptic curve digital signature algorithm (ECDSA). International journal of information security, 1(1), 36–63 [here].

<img src='graphics/g_sig_11.png' width="800px">

In [ ]:
# https://asecuritysite.com/hazmat/hashnew10
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography import exceptions
import binascii
import sys

private_key = ec.generate_private_key(ec.SECP384R1())

data = b"test"

if (len(sys.argv)>2):
	type=int(sys.argv[2])
if (len(sys.argv)>1):
	data=str(sys.argv[1]).encode()


private_key = ec.generate_private_key(ec.SECP256R1())


private_vals = private_key.private_numbers()
no_bits=private_vals.private_value.bit_length()

print (f"Private key value: {private_vals.private_value}. Number of bits {no_bits}")

public_key = private_key.public_key()
pub=public_key.public_numbers()
print ("Name of curve: ",pub.curve.name)

print ("Message: ",data.decode())
try:

  signature = private_key.sign(data,ec.ECDSA(hashes.SHA256()))
  print ("Good Signature: ",binascii.b2a_hex(signature).decode())
  public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))
except exceptions.InvalidSignature:
  print("A bad signature failed")
else:
  print("Good signature verified")

try:

  signature = private_key.sign(b"bad message",ec.ECDSA(hashes.SHA256()))
  print ("Bad Signature: ",binascii.b2a_hex(signature).decode())
  public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))
except exceptions.InvalidSignature:
  print("A bad signature failed")
else:
  print("Good signature verified")



pem = private_key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())

der = private_key.private_bytes(encoding=serialization.Encoding.DER,format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())



print ("\nPrivate key (PEM):\n",pem.decode())
print ("Private key (DER):\n",binascii.b2a_hex(der))

pem = public_key.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo)

der = public_key.public_bytes(encoding=serialization.Encoding.DER,format=serialization.PublicFormat.SubjectPublicKeyInfo)

print ("\nPublic key (PEM):\n",pem.decode())
print ("Public key (DER):\n",binascii.b2a_hex(der))

> Implement with the SECP521R1, and observe the differences in the keys sizes used.

> Implement the code with the BrainpoolP256R1 curve, and verify its operation.

> The bitcoin curve is secp256k1. Modify the program so that it uses secp256k1.

## EdDSA

With Ed25519 we use a private key to sign data, and then the public key can prove it. We use Curve 25519 for the generation of the public key and for the signing process. Curve 25519 uses a Montgomery curve of the form of y2=x3+ax2+x(modp). If we have a point P, we then find a point nP - where n is the number of times we add P. Curve 25519 takes the form of y^2=x^3+486662x^2+x(modp) and a base point at x=9. This gives a base point (G), and where we find the point nG for a given n value, and where n is the private key value, and nG is the public key point. Normally in Curve 25519 we only focus on the x-point, and do not need the (x,y) value.


<img src='graphics/g_sig_03.png' width="800px">

In [2]:
# https://asecuritysite.com/hazmat/hashnew11
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
from cryptography import exceptions
from cryptography.hazmat.primitives import serialization
import binascii
import sys

data=b"Test"

if (len(sys.argv)>1):
	data=str(sys.argv[1]).encode()

private_key = Ed25519PrivateKey.generate()
public_key = private_key.public_key()


public_key = private_key.public_key()

print ("Message: ",data.decode())
try:

  signature = private_key.sign(data)
  print ("Good Signature: ",binascii.b2a_hex(signature).decode())
  public_key.verify(signature, data)
except exceptions.InvalidSignature:
  print("A bad signature failed")
else:
  print("Good signature verified")

try:

  signature = private_key.sign(b"Bad data")
  print ("Bad Signature: ",binascii.b2a_hex(signature).decode())
  public_key.verify(signature, data)
except exceptions.InvalidSignature:
  print("A bad signature failed")
else:
  print("Good signature verified")



pem = private_key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())

der = private_key.private_bytes(encoding=serialization.Encoding.DER,format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())



print ("\nPrivate key (PEM):\n",pem.decode())
print ("Private key (DER):\n",binascii.b2a_hex(der))

pem = public_key.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo)

der = public_key.public_bytes(encoding=serialization.Encoding.DER,format=serialization.PublicFormat.SubjectPublicKeyInfo)

print ("\nPublic key (PEM):\n",pem.decode())
print ("Public key (DER):\n",binascii.b2a_hex(der))

Private key: 0x8429f9e6136fcfa883130853defd319d931e25207749de92b5e12a80c8e62619
Public key: (0xa4581b11e2aa0249274374216d8fb965517e45e6bcc337f3fc679c2bfd84dce7, 0x4716b0eaf5f748af3921aac8455e623b96b590453f313b4d200419a801c11267)


Random value:  23353093005829895849666617279986636640
rG:  (38430190155405327152400342621384391216588051326127513706497722898874838651319, 32551409443475257236910193737311399537798158570670164100113247023590451377060)


======Symmetric key========
Encryption key: 34425016396310312658614745921144437662843062008454472115656949568073393878924 34425016396310312658614745921144437662843062008454472115656949568073393878924
Encrypted:	 b'02c9b6ed4ffcf2bdf4f3118ecc232326'
Decrypted:	 Hello
